In [0]:
from pyspark.sql.functions import sequence, explode, date_format, col, year, month, dayofmonth, expr

In [0]:
# Define o range de datas desejado
data_inicio = "2020-01-01"
data_fim = "2030-12-31"

# Cria DataFrame com sequência de datas
df_calendario = (
    spark
    .range(1)
    .select(sequence(expr(f"to_date('{data_inicio}')"), expr(f"to_date('{data_fim}')")).alias("datas"))
    .select(explode(col("datas")).alias("data"))
)

# Dicionários para tradução de dias da semana e meses
dias_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}
meses_pt = {
    "January": "janeiro",
    "February": "fevereiro",
    "March": "março",
    "April": "abril",
    "May": "maio",
    "June": "junho",
    "July": "julho",
    "August": "agosto",
    "September": "setembro",
    "October": "outubro",
    "November": "novembro",
    "December": "dezembro"
}

# Cria UDFs para tradução
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(StringType())
def traduz_dia_semana(dia_en):
    return dias_semana_pt.get(dia_en, dia_en)

@udf(StringType())
def traduz_mes(mes_en):
    return meses_pt.get(mes_en, mes_en)

from pyspark.sql.functions import date_format

df_calendario = (
    df_calendario
    .withColumn("dia_semana_en", date_format(col("data"), "EEEE"))
    .withColumn("mes_en", date_format(col("data"), "MMMM"))
    .withColumn("dia_semana", traduz_dia_semana(col("dia_semana_en")))
    .withColumn("mes", traduz_mes(col("mes_en")))
    .withColumn("ano", year(col("data")))
    .withColumn("dia", dayofmonth(col("data")))
    .withColumn("data_completa", 
        expr("concat(dia_semana, ', ', dia, ' de ', mes, ' de ', ano)")
    )
    .withColumn("data_ddMMyyyy", date_format(col("data"), "dd-MM-yyyy"))
    .drop("dia_semana_en", "mes_en")
)

# Salva como tabela bronze.calendario
df_calendario.write.mode("overwrite").format("delta").saveAsTable("grao.bronze.calendario")